In [1]:


import numpy as np
import pandas as pd




In [2]:
import os
import torch
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import get_linear_schedule_with_warmup
from datasets import Dataset

# Config

In [3]:

model_name = 'microsoft/deberta-v3-small'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_path = "./results/full_training"

EPOCH = 7
FRAC = 1.0

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


# Load and sample dataset

In [10]:
train_df = pd.read_csv('/content/train_half.csv', on_bad_lines='skip')

# Drop rows with NaN values in the specified columns before sampling and mapping labels
train_df.dropna(subset=["winner_model_a", "winner_model_b", "winner_tie"], inplace=True)

# Explicitly convert winner columns to integer type after dropping NaNs
train_df["winner_model_a"] = pd.to_numeric(train_df["winner_model_a"], errors='coerce').fillna(0).astype(int)
train_df["winner_model_b"] = pd.to_numeric(train_df["winner_model_b"], errors='coerce').fillna(0).astype(int)
train_df["winner_tie"] = pd.to_numeric(train_df["winner_tie"], errors='coerce').fillna(0).astype(int)


df = train_df.sample(frac=FRAC, random_state=42).reset_index(drop=True)

def map_labels(row):
    if row["winner_model_a"] == 1:
        return 0
    elif row["winner_model_b"] == 1:
        return 1
    else:
        return 2

df["labels"] = df.apply(map_labels, axis=1).astype(int) # Ensure the labels are integers

/tmp/ipython-input-10-1595738943.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,

In [5]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df, test_size=0.1, stratify=df["labels"], random_state=42)

# Tokenization

In [11]:
import pyarrow as pa

def tokenize_batch(batch):
    text_a = [f"Prompt: {p} Response: {r}" for p, r in zip(batch["prompt"], batch["response_a"])]
    text_b = [f"Prompt: {p} Response: {r}" for p, r in zip(batch["prompt"], batch["response_b"])]
    return tokenizer(text_a, text_b, padding="max_length", truncation=True, max_length=256)

# Manually create PyArrow Table with explicit types
def df_to_arrow_table(df):
    # Ensure relevant columns exist and handle potential NaNs if not already done robustly
    # (Although previous step should have handled NaNs in winner columns,
    #  ensure prompt/response/id are strings and labels are int)
    df['id'] = df['id'].astype(str)
    df['prompt'] = df['prompt'].astype(str)
    df['response_a'] = df['response_a'].astype(str)
    df['response_b'] = df['response_b'].astype(str)
    df['labels'] = df['labels'].astype(int)


    table = pa.Table.from_arrays([
        pa.array(df['id'], type=pa.string()),
        pa.array(df['prompt'], type=pa.string()),
        pa.array(df['response_a'], type=pa.string()),
        pa.array(df['response_b'], type=pa.string()),
        pa.array(df['labels'], type=pa.int64())
    ], names=['id', 'prompt', 'response_a', 'response_b', 'labels'])
    return table


train_table = df_to_arrow_table(df_train)
val_table = df_to_arrow_table(df_val)

train_dataset = Dataset(train_table)
val_dataset = Dataset(val_table)

train_dataset = train_dataset.map(tokenize_batch, batched=True, batch_size=500)
val_dataset = val_dataset.map(tokenize_batch, batched=True, batch_size=500)


def clean(ds):
    return ds.remove_columns([col for col in ds.column_names if col not in ["input_ids", "attention_mask", "labels"]])

train_dataset = clean(train_dataset)
val_dataset = clean(val_dataset)

Map:   0%|          | 0/46660 [00:00<?, ? examples/s]

Map:   0%|          | 0/5185 [00:00<?, ? examples/s]

In [12]:
import numpy as np


input_ids_array = np.asarray(train_dataset["input_ids"], dtype=object)
print(input_ids_array.shape)


(46660, 256)


In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    return {"accuracy": acc, "precision": precision, "recall": recall, "eval_f1": f1}

# Load model

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = TrainingArguments(
    output_dir=model_path,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=EPOCH,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    learning_rate=5e-6,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir=f"./logs/full_training",
    logging_steps=3,
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    label_smoothing_factor=0.1,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

model.config.classifier_dropout_prob = 0.5
print("Load model successful!")

Load model successful!


# Training

In [ ]:
trainer.train()
trainer.save_model(model_path)

In [17]:
ENABLE_SUBMISSION = True

In [18]:
import pickle
import os

# Create the directory if it doesn't exist
os.makedirs("./saved", exist_ok=True)

# Save training_args using pickle
with open("./saved/training_args.pkl", "wb") as f:
    pickle.dump(training_args, f)

In [19]:
with open("./saved/training_args.pkl", "rb") as f:
    training_args = pickle.load(f)


In [20]:
import os
import pickle

os.makedirs("./saved", exist_ok=True)

# Save tokenizer and model
tokenizer.save_pretrained("./saved/tokenizer")
model.save_pretrained("./saved/model")

# Save training arguments
with open("./saved/training_args.pkl", "wb") as f:
    pickle.dump(training_args, f)

# Save datasets
with open("./saved/train_dataset.pkl", "wb") as f:
    pickle.dump(train_dataset, f)

with open("./saved/val_dataset.pkl", "wb") as f:
    pickle.dump(val_dataset, f)

print("✅ All components saved successfully.")


✅ All components saved successfully.


In [25]:
pip install gradio


In [22]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 802.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 457.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 627.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 832.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [34]:
import re
from collections import Counter
import gradio as gr # Import gradio

def extract_keywords(text):
    # Very simple keyword extractor (you can upgrade to spaCy or Rake)
    words = re.findall(r"\b\w+\b", text.lower())
    stop_words = set([
        "i", "you", "he", "she", "they", "we", "do", "does", "did", "am", "is", "are",
        "was", "were", "have", "has", "had", "the", "a", "an", "and", "or", "if", "then",
        "how", "many", "what", "when", "why", "where", "of", "to", "in", "on", "with", "by", "this", "that"
    ])
    keywords = [word for word in words if word not in stop_words]
    return Counter(keywords)

def predict_preference(prompt, response_a, response_b):
    try:
        # Clean & truncate
        prompt = str(prompt)[:1000]
        response_a = str(response_a)[:2000]
        response_b = str(response_b)[:2000]

        # Tokenize
        text_a = f"Prompt: {prompt} Response: {response_a}"
        text_b = f"Prompt: {prompt} Response: {response_b}"
        inputs = tokenizer(text_a, text_b, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Inference
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

        labels = ["Model A", "Model B", "Tie"]
        confidence = {label: round(float(prob), 4) for label, prob in zip(labels, probs)}
        predicted = labels[np.argmax(probs)]

        # ✨ Add Relevance Heuristic
        prompt_keywords = extract_keywords(prompt)
        response_keywords_a = extract_keywords(response_a)
        response_keywords_b = extract_keywords(response_b)

        match_a = sum((response_keywords_a & prompt_keywords).values())
        match_b = sum((response_keywords_b & prompt_keywords).values())

        # Boost the one with more matching keywords
        if match_a > match_b:
            confidence["Model A"] += 0.05
            confidence["Model B"] -= 0.05
        elif match_b > match_a:
            confidence["Model B"] += 0.05
            confidence["Model A"] -= 0.05

        # Clamp between 0 and 1
        for key in confidence:
            confidence[key] = round(min(max(confidence[key], 0.0), 1.0), 4)

        # Re-predict based on updated confidence
        predicted = max(confidence, key=confidence.get)

        return confidence, predicted

    except Exception as e:
        return {"Model A": 0.0, "Model B": 0.0, "Tie": 0.0}, f"❌ Error: {str(e)}"


# Gradio Interface
iface = gr.Interface(
    fn=predict_preference,
    inputs=[
        gr.Textbox(label="Prompt", lines=5, placeholder="Enter the prompt here..."),
        gr.Textbox(label="Response A", lines=6, placeholder="Paste Model A's response..."),
        gr.Textbox(label="Response B", lines=6, placeholder="Paste Model B's response...")
    ],
    outputs=[
        gr.Label(label="Confidence Scores"),
        gr.Textbox(label="Predicted Preference", lines=1)
    ],
    title="LLM Preference Classifier",
    description="Enter a prompt and two LLM responses. The model will predict which one is preferred (A, B, or Tie)."
)

# Launch with public shareable link
if __name__ == "__main__":
    iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f83d9eba44881ba5f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# import torch
# import numpy as np
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import gradio as gr

# # Load tokenizer and model from saved directory
# tokenizer = AutoTokenizer.from_pretrained("./saved/tokenizer")
# model = AutoModelForSequenceClassification.from_pretrained("./saved/model")
# model.eval()
# model.to("cuda" if torch.cuda.is_available() else "cpu")

# # Prediction function
# def predict_preference(prompt, response_a, response_b):
#     try:
#         # Truncate overly long inputs for safety
#         prompt = str(prompt)[:1000]
#         response_a = str(response_a)[:2000]
#         response_b = str(response_b)[:2000]

#         # Create input pairs
#         text_a = f"Prompt: {prompt} Response: {response_a}"
#         text_b = f"Prompt: {prompt} Response: {response_b}"
#         inputs = tokenizer(text_a, text_b, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
#         inputs = {k: v.to(model.device) for k, v in inputs.items()}

#         # Inference
#         with torch.no_grad():
#             outputs = model(**inputs)
#             logits = outputs.logits
#             probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

#         # Map predictions
#         labels = ["Model A", "Model B", "Tie"]
#         confidence = {label: round(float(prob), 4) for label, prob in zip(labels, probs)}
#         predicted = labels[np.argmax(probs)]

#         return confidence, predicted

#     except Exception as e:
#         return {"Model A": 0.0, "Model B": 0.0, "Tie": 0.0}, f"❌ Error: {str(e)}"

# # Gradio Interface
# iface = gr.Interface(
#     fn=predict_preference,
#     inputs=[
#         gr.Textbox(label="Prompt", lines=5, placeholder="Enter the prompt here..."),
#         gr.Textbox(label="Response A", lines=6, placeholder="Paste Model A's response..."),
#         gr.Textbox(label="Response B", lines=6, placeholder="Paste Model B's response...")
#     ],
#     outputs=[
#         gr.Label(label="Confidence Scores"),
#         gr.Textbox(label="Predicted Preference", lines=1)
#     ],
#     title="LLM Preference Classifier",
#     description="Enter a prompt and two LLM responses. The model will predict which one is preferred (A, B, or Tie)."
# )

# # Launch with public shareable link
# if __name__ == "__main__":
#     iface.launch(share=True)


In [38]:
import pandas as pd
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from scipy.special import softmax
import numpy as np # Import numpy

def tokenize_batch(batch):
    texts = [f"{p} {r_a} {r_b}" for p, r_a, r_b in zip(batch["prompt"], batch["response_a"], batch["response_b"])]
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=3,
        return_tensors=None
    )

if ENABLE_SUBMISSION:
    # 1. Load test dataframe
    test_df = pd.read_csv("/content/test.csv")
    test_ids = test_df["id"]

    # 2. Create HF dataset and tokenize
    test_dataset = Dataset.from_pandas(test_df)
    test_dataset = test_dataset.map(tokenize_batch, batched=True, remove_columns=test_dataset.column_names)

    # 3. Inspect columns and types after tokenization (Optional - for debugging)
    # print("Columns after tokenization:", test_dataset.column_names)
    # print("Features info:", test_dataset.features)


    # 4. Check tokenizer keys expected by your model
    tokenizer_keys = list(tokenizer.model_input_names)
    print("Tokenizer keys:", tokenizer_keys)

    # 5. Remove any unexpected columns explicitly if present
    cols_to_remove = [c for c in test_dataset.column_names if c not in tokenizer_keys]
    if cols_to_remove:
        test_dataset = test_dataset.remove_columns(cols_to_remove)
        print("Removed columns:", cols_to_remove)

    # Removed: 6. Set dataset format to torch tensors with only tokenizer keys
    # test_dataset.set_format(type="torch", columns=tokenizer_keys)

    # 7. Final check for the first item (Optional - for debugging, needs careful handling after set_format)
    # sample = test_dataset[0]
    # print("First sample tensors and shapes:")
    # for k in tokenizer_keys:
    #     print(f" - {k}: {sample[k].shape} dtype={sample[k].dtype}")


    # 8. Create dataloader and inference loop
    dataloader = DataLoader(test_dataset, batch_size=16)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainer.model.to(device)
    trainer.model.eval()

    all_logits = []
    with torch.no_grad():
        for batch in dataloader:
            # Manually convert batch items to tensors via numpy array
            batch = {k: torch.tensor(np.asarray(v)).to(device) for k, v in batch.items() if k in tokenizer_keys}
            outputs = trainer.model(**batch)
            logits = outputs.logits.detach().cpu()
            all_logits.append(logits)

    test_logits = torch.cat(all_logits).numpy()
    test_probs = softmax(test_logits, axis=-1)

    # Ensure test_ids is a simple list or array matching the length of predictions
    # Provide an explicit index to prevent ValueError
    submission = pd.DataFrame({
        "id": test_ids.tolist(), # Convert the Series to a list
        "winner_model_a": test_probs[:, 0].round(6),
        "winner_model_b": test_probs[:, 1].round(6),
        "winner_tie": test_probs[:, 2].round(6),
    }, index=np.arange(len(test_ids))) # Explicitly provide index

    submission_path = "/content/sample_submission.csv"
    submission.to_csv(submission_path, index=False)
    print(f"Submission saved to: {submission_path}")

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Tokenizer keys: ['input_ids', 'token_type_ids', 'attention_mask']
Submission saved to: /content/sample_submission.csv


In [40]:
submission

id  winner_model_a  winner_model_b  winner_tie
0   136060        0.358201        0.258613    0.383186
1   211333        0.130375        0.159642    0.709983
2  1233961        0.366251        0.260827    0.372922

In [59]:
import pandas as pd

# Your output data
data = {
    "id": [136060, 211333, 1233961],
    "winner_model_a": [0.3582009971141815, 0.1303749978542328, 0.36625099182128906],
    "winner_model_b": [0.2586129903793335, 0.15964199602603912, 0.2608270049095154],
    "winner_tie": [0.383186012506485, 0.7099829912185669, 0.37292200326919556],
}

submission = pd.DataFrame(data)
submission.to_csv("submission.csv", index=False)
from google.colab import files
files.download("submission.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task
Save the current notebook and all required files and folders into a downloadable archive.

In [44]:
import os
import zipfile

notebook_path = '/content/llm_preference_classifier.ipynb' # Assuming this is the notebook file name

# List of files and folders to include in the archive
paths_to_include = [
    notebook_path,
    '/content/train_half.csv',
    '/content/test.csv',
    './results/full_training',
    f"./logs/full_training",
    "./saved",
    "/content/sample_submission.csv",
]

output_zip_name = 'notebook_archive.zip'

In [45]:
with zipfile.ZipFile(output_zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for path in paths_to_include:
        if os.path.isfile(path):
            zipf.write(path, os.path.basename(path))
        elif os.path.isdir(path):
            for root, _, files in os.walk(path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, os.path.dirname(path))
                    zipf.write(file_path, arcname)

print(f"Archive '{output_zip_name}' created successfully.")

Archive 'notebook_archive.zip' created successfully.


**Reasoning**:
Use the Colab files API to download the created zip archive.



In [47]:
from google.colab import files

files.download('notebook_archive.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>